In [5]:
!pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 7.0 MB/s eta 0:00:00


In [17]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpmax

In [41]:
df = pd.read_csv("singfake_metadata.csv")

views_threshold = [0, 10000, 1000000, df['views'].max()]
likes_threshold = [0, 1000, 50000, df['likes'].max()]
comments_threshold = [0, 100, 5000, df['comments'].max()]

df['views_category'] = pd.cut(df['views'], bins=views_threshold, labels=['Low Views', 'Medium Views', 'High Views'])
df['likes_category'] = pd.cut(df['likes'], bins=likes_threshold, labels=['Low Likes', 'Medium Likes', 'High Likes'])
df['comments_category'] = pd.cut(df['comments'], bins=comments_threshold, labels=['Low Comments', 'Medium Comments', 'High Comments'])

df['bonafide_or_spoof'] = df['bonafide_or_spoof'].map({0: 'Real', 1: 'Fake'})
df['views_category'] = df['views_category'].astype(str)
df['likes_category'] = df['likes_category'].astype(str)
df['comments_category'] = df['comments_category'].astype(str)

df = df.fillna("Unknown")
transactions = df[['views_category', 'likes_category', 'comments_category', 'bonafide_or_spoof']].values.tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
print(df_encoded.sum())

Fake               631
High Comments       50
High Likes         113
High Views         276
Low Comments       793
Low Likes          792
Low Views          422
Medium Comments    424
Medium Likes       378
Medium Views       587
Real               654
nan                 20
dtype: int64


In [43]:
frequent_items = fpmax(df_encoded, min_support=0.1, use_colnames=True)
print(frequent_items)

file_path = "frequent_items.txt"
with open(file_path, "w") as f:
    f.write(frequent_items.to_string(index=False))

    support                                           itemsets
0  0.119066  (High Views, Medium Likes, Real, Medium Comments)
1  0.110506                 (Medium Views, Medium Likes, Real)
2  0.133852      (Medium Views, Medium Likes, Medium Comments)
3  0.275486         (Low Likes, Low Comments, Low Views, Fake)
4  0.121401      (Medium Views, Low Likes, Low Comments, Fake)
5  0.140856      (Medium Views, Low Comments, Real, Low Likes)
